# Анализ файлов выгрузки из Keys.so

In [1]:
import pandas as pd

## Объявляем пути к файлам с данными, константные значения, а также функции возвращающие другие заданные значения

In [2]:
organic_keys_file = './data/citysee.ru.organic.keys.msk'

# Ctr по данным исследования OverLead https://overlead.me/blog/ctr-na-vydache-yandeksa-ot-2019-goda/
def get_ctr15(pos):
    return {
        pos <= 0:   0.0,
        pos == 1:   27.42,
        pos == 2:   17.38,
        pos == 3:   10.66,
        pos == 4:   6.69,
        pos == 5:   4.97,
        pos == 6:   4.19,
        pos == 7:   4.39,
        pos == 8:   4.23,
        pos == 9:   4.41,
        pos == 10:   2.83,
        pos == 11:   1.68,
        pos == 12:   1.93,
        pos == 13:   2.52,
        pos == 14:   3.45,
        pos == 15:   3.75,
        pos > 15:   0.0
    }[True]

def get_ctr10(pos):
    return {
        pos <= 0:   0.0,
        pos == 1:   31.19,
        pos == 2:   11.20,
        pos == 3:   4.69,
        pos == 4:   3.23,
        pos == 5:   3.05,
        pos == 6:   3.86,
        pos == 7:   3.93,
        pos == 8:   2.61,
        pos == 9:   1.85,
        pos == 10:   1.32,
        pos > 10:   0.0
    }[True]

In [3]:
organic_keys_df = pd.read_csv(organic_keys_file + '.csv', sep=';', usecols=[0, 1, 2, 3, 4],
                              names=['Query', 'Page', 'Position', '[Wordstat]', '[!Wordstat]'], header=0)

In [4]:
organic_keys_df.head()

,Query,Page,Position,[Wordstat],[!Wordstat]
0,достопримечательности бонна на немецком,http://citysee.ru/europe/zapadnaya/germaniya/b...,1,4,1
1,витебск достопримечательности и туристические ...,http://citysee.ru/europe/vostochnaya/belarusia...,1,25,20
2,звенигород достопримечательности зимой,http://citysee.ru/russia/tsfo/moskovskaya-obla...,1,44,7
3,звенигород достопримечательности маршрут,http://citysee.ru/russia/tsfo/moskovskaya-obla...,1,11,1
4,главные достопримечательности кремля,http://citysee.ru/russia/gorod-federalnogo-zna...,1,199,1


In [5]:
organic_keys_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Query        20000 non-null  object
 1   Page         20000 non-null  object
 2   Position     20000 non-null  int64 
 3   [Wordstat]   20000 non-null  int64 
 4   [!Wordstat]  20000 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 781.4+ KB


In [6]:
organic_keys_df.describe()

,Position,[Wordstat],[!Wordstat]
count,20000.000000,2.000000e+04,20000.000000
mean,20.323650,1.200039e+04,56.084000
std,12.934522,1.603985e+05,544.556433
min,1.000000,1.000000e+00,1.000000
25%,10.000000,2.200000e+01,2.000000
50%,17.000000,7.700000e+01,6.000000
75%,29.000000,3.882500e+02,17.000000
max,50.000000,1.176360e+07,27497.000000


In [7]:
organic_keys_df['Traffic10'] = organic_keys_df['[!Wordstat]'] * organic_keys_df['Position'].apply(
    lambda x: get_ctr10(x) / 100)
organic_keys_df['Traffic15'] = organic_keys_df['[!Wordstat]'] * organic_keys_df['Position'].apply(
    lambda x: get_ctr15(x) / 100)

organic_keys_df['QueryCount'] = 1
organic_keys_df = organic_keys_df.drop('Position', 1)
organic_keys_df.head()

,Query,Page,[Wordstat],[!Wordstat],Traffic10,Traffic15,QueryCount
0,достопримечательности бонна на немецком,http://citysee.ru/europe/zapadnaya/germaniya/b...,4,1,0.3119,0.2742,1
1,витебск достопримечательности и туристические ...,http://citysee.ru/europe/vostochnaya/belarusia...,25,20,6.2380,5.4840,1
2,звенигород достопримечательности зимой,http://citysee.ru/russia/tsfo/moskovskaya-obla...,44,7,2.1833,1.9194,1
3,звенигород достопримечательности маршрут,http://citysee.ru/russia/tsfo/moskovskaya-obla...,11,1,0.3119,0.2742,1
4,главные достопримечательности кремля,http://citysee.ru/russia/gorod-federalnogo-zna...,199,1,0.3119,0.2742,1


In [8]:
organic_keys_df.groupby('Page').sum().sort_values('Traffic10', ascending=False)

,[Wordstat],[!Wordstat],Traffic10,Traffic15,QueryCount
Page,,,,,
http://citysee.ru/europe/vostochnaya/belarusian/brest-dostoprimechatelnosti-goroda.html,8135333,25313,635.8026,1089.9059,257
http://citysee.ru/russia/pfo/respublika-tatarstan/almetevsk-dostoprimechatelnosti-goroda.html,3239882,18054,528.0956,874.3158,149
http://citysee.ru/russia/szfo/murmanskaya-oblast/kirovsk-murmanskaya-oblast-dostoprimechatelnosti.html,827259,16455,498.3522,1017.9627,97
http://citysee.ru/russia/gorod-federalnogo-znacheniya/teatryi-sankt-peterburga-spisok.html,3310074,11393,340.2045,483.1849,249
http://citysee.ru/russia/szfo/murmanskaya-oblast/apatityi-dostoprimechatelnosti-goroda.html,861020,10674,325.2489,524.0089,59
...,...,...,...,...,...
http://citysee.ru/russia/gorod-federalnogo-znacheniya/muzei-moskvyi-v-kotoryih-nuzhno-obyazatelno-pobyivat.html,241251,6612,0.0000,21.1661,104
http://citysee.ru/russia/yufo/respublika-kryim/yalta-chto-posmotret-i-kuda-shodit.html,4779,299,0.0000,2.2680,24
http://citysee.ru/russia/szfo/leningradskaya-oblast/vyborg-dostoprimechatelnosti-chto-posmotret-za-1-den.html,5969,1103,0.0000,0.3146,45


In [9]:
organic_keys_df.groupby('Page').sum().sort_values('Traffic10', ascending=False).to_csv(
    organic_keys_file + '.traffic.csv', sep=';', decimal=',', encoding='utf-8')